In [1]:
### xgboost xgboost se quitaron features de promedio sin HT
0.60179 
sumit 4



In [2]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [4]:
path_dataset = 'train.csv'
train = pd.read_csv(path_dataset)

In [3]:
path_dataset = 'test.csv'
test = pd.read_csv(path_dataset)

In [15]:
#Modificar dimensiones predeterminadas de visibilidad de columnas y filas
x = 52
pd.set_option('display.max_columns', x) #Maximo de columnas a ver
pd.set_option('display.max_rows', x) #Maximo de Filas a ver

In [5]:
train['Stage']= (train['Stage'] == 'Closed Won') +0

In [9]:
for column in train.columns:
    if train[column].dtype == 'int32':
        train[column] = train[column].astype('float64')
print(train.dtypes)

Opportunity_ID                   int64
Region                          object
Billing_Country                 object
Account_Type                    object
Opportunity_Type                object
Account_Owner                   object
Account_Name                    object
Conversión_USD                   int64
Delivery_Quarter                object
TRF                              int64
Stage                          float64
promedio_preciov_region        float64
promedio_preciov_paises        float64
promedio_preciov_Account       float64
promedio_preciov_Oportunity    float64
promedio_preciov_vendedor      float64
Wilsonxproporcion_region       float64
Wilsonxproporcion_paises       float64
Wilsonxproporcion_vendedor     float64
dtype: object


In [ ]:
['Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name','Delivery_Quarter']

In [6]:
train.shape

(16942, 19)

In [7]:
test.shape

(2551, 18)

In [10]:
### Target meaning encoding

In [11]:
# Import KFold
from sklearn.model_selection import KFold

In [23]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
   
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [24]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [25]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [26]:
train.columns

Index(['Opportunity_ID', 'Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Stage', 'promedio_preciov_region',
       'promedio_preciov_paises', 'promedio_preciov_Account',
       'promedio_preciov_Oportunity', 'promedio_preciov_vendedor',
       'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor'],
      dtype='object')

In [27]:
columns = ['Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name','Delivery_Quarter']
for column in columns:     
    train[column], test[column] = mean_target_encoding(train=train,
                                                                test=test,
                                                                target='Stage',
                                                                categorical=column,
                                                                alpha=10)

<ipython-input-24-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16942 entries, 0 to 16941
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Opportunity_ID               16942 non-null  int64  
 1   Region                       16942 non-null  float64
 2   Billing_Country              16942 non-null  float64
 3   Account_Type                 16942 non-null  float64
 4   Opportunity_Type             16942 non-null  float64
 5   Account_Owner                16942 non-null  float64
 6   Account_Name                 16942 non-null  float64
 7   Conversión_USD               16942 non-null  int64  
 8   Delivery_Quarter             16942 non-null  float64
 9   TRF                          16942 non-null  int64  
 10  Stage                        16942 non-null  float64
 11  promedio_preciov_region      16942 non-null  float64
 12  promedio_preciov_paises      16942 non-null  float64
 13  promedio_preciov

In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Opportunity_ID               2551 non-null   int64  
 1   Region                       2551 non-null   float64
 2   Billing_Country              2551 non-null   float64
 3   Account_Type                 2551 non-null   float64
 4   Opportunity_Type             2551 non-null   float64
 5   Account_Owner                2551 non-null   float64
 6   Account_Name                 2551 non-null   float64
 7   Conversión_USD               2551 non-null   int64  
 8   Delivery_Quarter             2551 non-null   float64
 9   TRF                          2551 non-null   int64  
 10  promedio_preciov_region      2551 non-null   float64
 11  promedio_preciov_paises      2534 non-null   float64
 12  promedio_preciov_Account     2551 non-null   float64
 13  promedio_preciov_O

In [30]:
### Modelos

In [31]:
X = train.drop(['Stage'], axis=1)
y = train['Stage']

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
from sklearn.metrics import log_loss

In [34]:
xg_reg = xgb.XGBClassifier(objective ='reg:squaredlogerror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [35]:
xg_reg.fit(X_train,y_train)

C:\Users\Julia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=4, num_parallel_tree=1,
              objective='reg:squaredlogerror', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [36]:
preds = xg_reg.predict_proba(X_test)

In [37]:
preds

array([[0.2527727 , 0.7472273 ],
       [0.895456  , 0.10454397],
       [0.895456  , 0.10454397],
       ...,
       [0.26887232, 0.7311277 ],
       [0.25981343, 0.7401866 ],
       [0.24191505, 0.75808495]], dtype=float32)

In [38]:
X_test

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
9900,6260,0.455400,0.463823,0.709296,0.686869,0.589056,0.710135,231918,0.514946,1,422783.847183,432143.813675,116533.133558,1.024004e+05,684165.342071,0.445687,0.448366,0.561500
1006,540,0.455400,0.463823,0.541830,0.216479,0.274424,0.340130,4126027,0.549676,11,422783.847183,0.000000,0.000000,0.000000e+00,0.000000,0.445687,0.448366,0.250712
4878,2898,0.585016,0.434141,0.296494,0.216479,0.486625,0.513271,4682427,0.514946,12,108927.857068,0.000000,0.000000,0.000000e+00,0.000000,0.560352,0.391691,0.453016
2604,1451,0.531415,0.275834,0.709296,0.531780,0.285128,0.513271,302241,0.565722,1,246888.239174,0.000000,0.000000,0.000000e+00,0.000000,0.514817,0.255594,0.262807
2415,1356,0.683098,0.684943,0.541830,0.686869,0.651691,0.482957,143,0.565722,0,35814.954559,0.000000,0.000000,0.000000e+00,0.000000,0.662223,0.666422,0.631065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15404,9824,0.455400,0.463823,0.541830,0.686869,0.352513,0.508587,31680,0.629957,0,422783.847183,432143.813675,125034.170409,1.024004e+05,259652.060345,0.445687,0.448366,0.322673
8185,5124,0.531415,0.566547,0.709296,0.686869,0.540692,0.864422,61344,0.565722,0,246888.239174,0.000000,0.000000,0.000000e+00,0.000000,0.514817,0.513767,0.514016
7694,4753,0.531415,0.566547,0.296494,0.216479,0.540692,0.657589,1894371,0.629957,4,246888.239174,177583.146032,958150.887348,1.629803e+06,184917.073702,0.514817,0.513767,0.514016
2667,1485,0.683098,0.684943,0.541830,0.686869,0.651691,0.482957,14,0.565722,0,35814.954559,34454.245862,125034.170409,1.024004e+05,10595.669831,0.662223,0.666422,0.631065


In [40]:
# Import log_loss from sklearn
from sklearn.metrics import log_loss

In [41]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

Sklearn LogLoss: 0.21124


In [42]:
preds = preds[:, 1]

In [43]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

Sklearn LogLoss: 0.21124


In [ ]:
###Predict en test

In [44]:
preds_test = xg_reg.predict_proba(test)

In [45]:
preds_test = preds_test[:, 1]

In [46]:
test['Target']=preds_test

In [47]:
sub= test[['Opportunity_ID','Target']]

In [48]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [50]:
sub1.shape

(1567, 2)

In [49]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission3.csv', index=False)

In [ ]:
Your submission scored 0.80485, which is not an improvement of your best score. Keep trying!

In [52]:
features = ['Opportunity_ID', 'Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF','Wilsonxproporcion_region', 'Wilsonxproporcion_paises', 'Wilsonxproporcion_vendedor']

In [70]:
xg_reg4 = xgb.XGBClassifier(objective ='reg:squaredlogerror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [75]:
xg_reg4.fit(X_train[features],y_train)

C:\Users\Julia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=4, num_parallel_tree=1,
              objective='reg:squaredlogerror', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [76]:
preds = xg_reg4.predict(X_test[features])

In [77]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

Sklearn LogLoss: 7.73536


In [78]:
preds_test = xg_reg4.predict_proba(test[features])

In [79]:
preds_test = preds_test[:, 1]

In [80]:
test['Target']=preds_test

In [81]:
sub= test[['Opportunity_ID','Target']]

In [82]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [83]:
sub1.shape

(1567, 2)

In [84]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission4.csv', index=False)